# 0. DEPENDENCIES

Fix for Jupyter Notebook imports:

In [1]:

import os
import sys

print(os.getcwd())

sys.path.append("S:\Dropbox\\000 - CARND\CarND-T1-P5-Vehicle-Detection")

for path in sys.path: print(path)
    

S:\Dropbox\000 - CARND\CarND-T1-P5-Vehicle-Detection\src\notebooks

C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\python35.zip
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\DLLs
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\cycler-0.10.0-py3.5.egg
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\IPython\extensions
C:\Users\gmzco\.ipython
S:\Dropbox\000 - CARND\CarND-T1-P5-Vehicle-Detection


Remove the additional entry if needed:

In [2]:

# sys.path = sys.path[:-1]

for path in sys.path: print(path)
    


C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\python35.zip
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\DLLs
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\cycler-0.10.0-py3.5.egg
C:\Users\gmzco\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\IPython\extensions
C:\Users\gmzco\.ipython
S:\Dropbox\000 - CARND\CarND-T1-P5-Vehicle-Detection


Load all dependencies:

In [3]:

import cv2
import glob
import numpy as np

from importlib import reload
from sklearn.externals import joblib
from moviepy.editor import VideoFileClip

import src.helpers.constants as C
import src.helpers.finder as FIND
import src.helpers.overlays as OV

# RELOAD:

reload(C)
reload(FIND)
reload(OV)


<module 'src.helpers.overlays' from 'S:\\Dropbox\\000 - CARND\\CarND-T1-P5-Vehicle-Detection\\src\\helpers\\overlays.py'>

# 1. LOAD THE CLASSIFIER AND GET THE WINDOWS

In [4]:

clf = joblib.load("../../output/models/classifier_augmented_nocars_2.pkl")
# clf = joblib.load("../../output/models/classifier_augmented_hls.pkl")
# clf = joblib.load("../../output/models/classifier.pkl")
# clf = joblib.load("../../output/models/classifier_svc.pkl")

HEIGHT = 720
WIDTH = 1280

WINDOWS = C.WINDOWS(WIDTH)

print("TOTAL WINDOWS = {}".format(len(C.WINDOWS(WIDTH))))


TOTAL WINDOWS = 378


# 2. CREATE THE VIDEO PROCESSING PIPELINE

First, let's define a function that will process each individual frame:

In [5]:
from collections import deque

class VehicleTracker():
    
    def __init__(self, size=8):
        self.size = size
        self.reset()
        
    def reset(self):
        self.items = deque(maxlen=self.size)
        
    def push(self, detections):
        self.items.append(detections)

        
vehicle_tracker = VehicleTracker()

            
def process_frame(img): # RGB
    img_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    detections = FIND.find_cars(img_hls, WINDOWS, clf)
    
    if len(detections) > 0:
        vehicle_tracker.push(detections)
    
    all_detections = list(vehicle_tracker.items)
    all_detections = np.concatenate(np.array(all_detections)) if len(all_detections) > 0 else []
    
    heatmap = OV.heatmap_boxes(all_detections, HEIGHT, WIDTH)
        
    heatmap = OV.heatmap_threshold(heatmap, 2)

    #heatmap = OV.heatmap_boxes(detections, HEIGHT, WIDTH)
    #heatmap = OV.heatmap_threshold(heatmap, 2)
    
    boxes = FIND.find_boxes(heatmap)
    
    return OV.draw_boxes(img, boxes)
            
    
index = 1
    
def extract_false_positives(img): # RGB
    img_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    detections = FIND.find_cars(img_hls, WINDOWS, clf)
    
    global index
    
    for box in detections:
        start, end = box
        x0, y0 = start
        x1, y1 = end
        
        if x1 < 0.6 * WIDTH:
            tile = cv2.resize(img[y0:y1, x0:x1], (64, 64))
            tile = cv2.cvtColor(tile, cv2.COLOR_RGB2BGR)
            cv2.imwrite("../../input/images/dataset/non-vehicles/falses-full/falsef{}.png".format(index), tile)
    
            index += 1
    
    return img


Next, let's define a function that will take each video and process all its frames with the function above:

In [6]:

def process_video(videosDir, videoName, outputDir):
    print("--------\n\nProcessing %s:" % videoName)
    
    inputClip = VideoFileClip(videosDir + videoName)
    outputClip = inputClip.fl_image(process_frame)
    %time outputClip.write_videofile(outputDir + "NEW " + videoName, audio=False)
    
    return outputClip
    

# 3. PROCESS THE VIDEOS

Let's see how many videos we have and devide which ones we want to process now:

In [ ]:

outputDir = "../../output/videos/"
videosDir = "../../input/videos/"
videosNames = os.listdir(videosDir)

videosNames = ["001 - Test Video (Sample).mp4", "002 - Test Video (0-3s).mp4", "003 - Test Video (25-30s).mp4"]
# videosNames = ["002 - Test Video (0-3s).mp4", "003 - Test Video (25-30s).mp4"]
# videosNames = ["004 - Project Video (Full).mp4"]

print("Available videos:\n")

for videoName in videosNames: print(videoName)
    

Available videos:

001 - Test Video (Sample).mp4
002 - Test Video (0-3s).mp4
003 - Test Video (25-30s).mp4


In [ ]:

for videoName in videosNames:
    vehicle_tracker.reset()
    
    output = process_video(videosDir, videoName, outputDir)
    

--------

Processing 001 - Test Video (Sample).mp4:
[MoviePy] >>>> Building video ../../output/videos/NEW 001 - Test Video (Sample).mp4
[MoviePy] Writing video ../../output/videos/NEW 001 - Test Video (Sample).mp4


 97%|███████████████████████████████████████████████████████████████████████████████▉  | 38/39 [01:18<00:01,  1.88s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../../output/videos/NEW 001 - Test Video (Sample).mp4 

Wall time: 1min 19s
--------

Processing 002 - Test Video (0-3s).mp4:
[MoviePy] >>>> Building video ../../output/videos/NEW 002 - Test Video (0-3s).mp4
[MoviePy] Writing video ../../output/videos/NEW 002 - Test Video (0-3s).mp4


 82%|███████████████████████████████████████████████████████████████████               | 63/77 [01:57<00:26,  1.86s/it]